In [1]:
import pyedflib
import numpy as np

from scipy import signal
import matplotlib.pyplot as plt
import csv
import pandas as pd
import itertools as iter

In [2]:
#read dataset
csv_file = 'mros-visit1-dataset-0.3.0.csv'
df = pd.read_csv(csv_file, dtype={"nsrrid": str, "poordi4": int}, low_memory=False)
AHI = df[['nsrrid', 'poordi4']] #select only two columns

#select only 50 patients from each group (normal/abnormal)
n_data = 100
group1 = AHI[AHI.poordi4 < 5][0:n_data/2] #group1 : normal
group1['class'] = 0
group2 = AHI[AHI.poordi4 >= 30][0:n_data/2] #group2 : OSA patient
group2['class'] = 1
all_data = pd.concat([group1, group2])

In [3]:
## get ECGs
ECGs = None

#set params
sampling_rate = 512 #number of samplings per minute
second_hour_start = sampling_rate * 60
fifth_hour_end = sampling_rate * 60 * 5
length = fifth_hour_end-second_hour_start
ECGs = np.zeros(shape=(n_data, length))

for i, it in enumerate(iter.izip(all_data['nsrrid'], all_data['class'])):
    try:
        print 'round', i
        f_name = "../mros/polysomnography/edfs/visit1/mros-visit1-" + it[0].lower() + ".edf"
        f = pyedflib.EdfReader(f_name)

        '''
        #Uncomment to see shape of data
        if i == 0:
            n = f.signals_in_file
            signal_labels = f.getSignalLabels()
            print "This file includes", n, "signals :"
            print "(list of signal : number of samples in each channel)"
            for i, s in enumerate(signal_labels):
                print '\t', s, ':', f.getNSamples()[i]
        '''

        #select only ECG
        ecg_L = f.readSignal(9)

        #select only data from 2nd to 5th hours
        ecg_L_4hrs = ecg_L[second_hour_start : fifth_hour_end]
        ECGs[i] = ecg_L_4hrs + it[1]
        
        #write selected period of signal to file
        fw_name = "./ECGs/visit1/mros-visit1-" + it[0].lower() + "-4hours.edf"
        fw = pyedflib.EdfWriter(fw_name, 1)
        fw.writeSamples([ecg_L_4hrs])
        print "Select ECG data only from 2nd to 5th hours =", length, "samplings from all", len(ecg_L), "samplings."
    
    except Exception:
        print 'FAIL at', f_name, Exception
    finally:
        f._close()
        fw.close()
    
print ECGs.shape

round 0
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 16128000 samplings.
round 1
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 22256640 samplings.
round 2
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 19046400 samplings.
round 3
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 20259840 samplings.
round 4
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 16204800 samplings.
round 5
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 22195200 samplings.
round 6
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 19184640 samplings.
round 7
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 22256640 samplings.
round 8
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 23040000 samplings.
round 9
Select ECG data only from 2nd to 5th hours = 122880 samplings from all 17326080 samplings.
round 10
S

FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0591.edf <type 'exceptions.Exception'>
round 82
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0676.edf <type 'exceptions.Exception'>
round 83
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0688.edf <type 'exceptions.Exception'>
round 84
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0699.edf <type 'exceptions.Exception'>
round 85
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0725.edf <type 'exceptions.Exception'>
round 86
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0728.edf <type 'exceptions.Exception'>
round 87
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0738.edf <type 'exceptions.Exception'>
round 88
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0739.edf <type 'exceptions.Exception'>
round 89
FAIL at ../mros/polysomnography/edfs/visit1/mros-visit1-aa0742.edf <type 'exceptions.Exception'>
round 90
FAIL at ../mros/polysomnography/edfs/visit1/mr

In [ ]:
#notch digital filter (from https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.iirnotch.html)
fs = sampling_rate  # Sample frequency (Hz)
f0 = 60.0  # Frequency to be removed from signal (Hz) => because of the US electrical 
Q = 30.0  # Quality factor
w0 = f0/(fs/2)  # Normalized Frequency
b, a = signal.iirnotch(w0, Q)
w, h = signal.freqz(b, a)

# Generate frequency axis
freq = w*fs/(2*np.pi)
print len(w), len(h), len(freq)

In [ ]:
f._close()
fw.close()